# Libraries import

In [13]:
# Data manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Model persistence
from joblib import dump, load

# Datasets

# Machine Learning
from sklearn.neighbors import 


from sklearn.utils import shuffle   
from sklearn.utils.fixes import loguniform
from sklearn.pipeline import Pipeline   
from sklearn.feature_extraction.text import CountVectorizer, \
                                            TfidfTransformer
from sklearn.metrics import classification_report, \
                            confusion_matrix, \
                            plot_confusion_matrix
from sklearn.model_selection import GridSearchCV, \
                                    RandomizedSearchCV, \
                                    train_test_split 


# Nearest Neighbors Classification

# Nearest Neighbors Regression